<a href="https://colab.research.google.com/github/SidharthGanesan/SG_Pandas/blob/main/WebScrapper_ASP.NetWebsite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests pandas beautifulsoup4 lxml

In [ ]:
import os
import time
import datetime as dt
from pathlib import Path

import requests
import pandas as pd
from bs4 import BeautifulSoup

BASE_MENU_URL = "https://fcainfoweb.nic.in/reports/report_menu_web.aspx"
REPORT_PAGE_URL = "https://fcainfoweb.nic.in/reports/Report_daily1_Web_New.aspx"  # landing page after "Get Data"
OUT_DIR = Path("/content/sample_data/GovDatasetonPrice")
OUT_DIR.mkdir(parents=True, exist_ok=True)

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/124.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
}

# ---- Helpers ---------------------------------------------------------------

def extract_hidden_fields(html: str) -> dict:
    """
    Pull all hidden <input> fields so we always carry fresh ASP.NET state forward.
    """
    soup = BeautifulSoup(html, "html.parser")
    data = {}
    for inp in soup.select("input[type=hidden]"):
        name = inp.get("name")
        if not name:
            continue
        data[name] = inp.get("value", "")
    # Some pages enforce presence of __VIEWSTATEENCRYPTED even if empty:
    data.setdefault("__VIEWSTATEENCRYPTED", "")
    # Keep a few common content controls defaulted if present:
    # (these keys exist on the page; leaving them empty is fine)
    data.setdefault("ctl00_MainContent_ToolkitScriptManager1_HiddenField", "")
    return data


def postback(session: requests.Session, url: str, state: dict, updates: dict) -> requests.Response:
    """
    Merge current ASP.NET hidden state with control updates and POST.
    Returns the response; caller should immediately refresh `state` using the response HTML.
    """
    payload = {**state, **updates}
    # ASP.NET expects EVENTARGUMENT even if empty when EVENTTARGET is present
    if "__EVENTTARGET" in payload and "__EVENTARGUMENT" not in payload:
        payload["__EVENTARGUMENT"] = ""
    resp = session.post(url, data=payload, headers=HEADERS, allow_redirects=True, timeout=60)
    resp.raise_for_status()
    return resp


def parse_report_table(html: str) -> pd.DataFrame:
    """
    Parse the Daily Prices table(s) from the report page.
    We'll pick the largest table as the main dataset.
    """
    # There can be multiple small header tables; choose the widest table with >= 5 columns.
    tables = pd.read_html(html)  # requires lxml
    if not tables:
        return pd.DataFrame()
    tables = [t for t in tables if t.shape[1] >= 5]
    if not tables:
        return pd.DataFrame()
    df = max(tables, key=lambda t: (t.shape[0], t.shape[1]))
    if df.empty:  # Add check for empty DataFrame
        return pd.DataFrame()
    # Normalize column names
    df.columns = [str(c).strip().replace("\n", " ") for c in df.columns]
    return df


def get_daily_prices_for_date(session: requests.Session, date_obj: dt.date) -> pd.DataFrame:
    """
    With an already-initialized session on the menu flow, set the date and fetch the report page,
    then parse and return the main DataFrame.
    """
    # 1) Load menu (fresh hidden fields)
    r0 = session.get(BASE_MENU_URL, headers=HEADERS, timeout=60)
    r0.raise_for_status()
    state = extract_hidden_fields(r0.text)

    # 2) Select: Report Type = Retail, Language = English, Report kind = Price report
    #    Many ASP.NET pages require both setting the control value AND indicating the EVENTTARGET that triggered postback.
    #    The radio list typically uses `$0` index suffix for the first item (Price report).
    step1_updates = {
        "__EVENTTARGET": "ctl00$MainContent$Rbl_Rpt_type$0",
        "ctl00$MainContent$Ddl_Rpt_type": "Retail",
        "ctl00$MainContent$ddl_Language": "English",
        "ctl00$MainContent$Rbl_Rpt_type": "Price report",
    }
    r1 = postback(session, BASE_MENU_URL, state, step1_updates)
    state = extract_hidden_fields(r1.text)

    # 3) Select: Report Option = Daily Prices
    step2_updates = {
        "__EVENTTARGET": "ctl00$MainContent$Ddl_Rpt_Option0",
        "ctl00$MainContent$Ddl_Rpt_Option0": "Daily Prices",
    }
    r2 = postback(session, BASE_MENU_URL, state, step2_updates)
    state = extract_hidden_fields(r2.text)

    # 4) Set date and click "Get Data"
    #    The control names below have been observed historically; if they change, view-source and update.
    date_str = date_obj.strftime("%d/%m/%Y")  # dd/MM/yyyy
    step3_updates = {
        "__EVENTTARGET": "",
        "ctl00$MainContent$Txt_FrmDate": date_str,
        "ctl00$MainContent$btn_getdata1": "Get Data",
    }
    r3 = postback(session, BASE_MENU_URL, state, step3_updates)

    # After successful "Get Data", the app usually serves/redirects to Report_daily1_Web_New.aspx using the same session.
    # Some environments return the report directly in r3; to be safe, request the known report URL.
    r_report = session.get(REPORT_PAGE_URL, headers=HEADERS, timeout=60)
    r_report.raise_for_status()

    # Parse
    df = parse_report_table(r_report.text)
    if df.empty:
        # Fallback: try parsing r3 if direct page didn't respond as expected
        df = parse_report_table(r3.text)

    # Attach metadata
    if not df.empty:
        df["ReportDate"] = date_obj.isoformat()
        df["Segment"] = "Retail"
        df["Report"] = "Daily Prices"
    return df


# ---- Main ------------------------------------------------------------------

def main():
    start = dt.date(2020, 3, 1)
    end = dt.date(2020, 3, 31)
    session = requests.Session()
    session.headers.update(HEADERS)

    all_days = []
    d = start
    while d <= end:
        print(f"Fetching {d.isoformat()} ...")
        try:
            df = get_daily_prices_for_date(session, d)
            if df.empty:
                print(f"  ⚠ No rows parsed for {d.isoformat()}. Saving HTML for inspection.")
                # Save raw HTML for debugging
                with open(OUT_DIR / f"debug_{d.isoformat()}.html", "w", encoding="utf-8") as f:
                    # Try to capture the last response content we can
                    f.write(session.get(REPORT_PAGE_URL, headers=HEADERS, timeout=60).text)
            else:
                # Save per-day CSV
                out_file = OUT_DIR / f"daily_prices_retail_{d.isoformat()}.csv"
                df.to_csv(out_file, index=False, encoding="utf-8-sig")
                print(f"  ✅ Saved {out_file.name} ({len(df)} rows)")
                all_days.append(df)
        except requests.HTTPError as e:
            print(f"  ❌ HTTP error on {d.isoformat()}: {e}")
        except Exception as e:
            print(f"  ❌ Unexpected error on {d.isoformat()}: {e}")
        time.sleep(1.0)  # be polite
        d += dt.timedelta(days=1)

    # Optional: combined CSV
    if all_days:
        combined = pd.concat(all_days, ignore_index=True)
        combined.to_csv(OUT_DIR / "daily_prices_retail_2020-03-01_to_2020-03-31.csv", index=False, encoding="utf-8-sig")
        print(f"\nCombined CSV written with {len(combined)} total rows.")

        combined.to_excel(OUT_DIR / "daily_prices_retail_2020-03-01_to_2020-03-31.xlsx", index=False)
        print(f"\nCombined CSV written with {len(combined)} total rows.")

if __name__ == "__main__":
    main()

Fetching 2020-03-01 ...


/tmp/ipython-input-458471964.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)  # requires lxml


  ✅ Saved daily_prices_retail_2020-03-01.csv (135 rows)
Fetching 2020-03-02 ...


/tmp/ipython-input-458471964.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)  # requires lxml


  ✅ Saved daily_prices_retail_2020-03-02.csv (135 rows)
Fetching 2020-03-03 ...


...

  ✅ Saved daily_prices_retail_2020-03-29.csv (135 rows)
Fetching 2020-03-30 ...


/tmp/ipython-input-458471964.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)  # requires lxml


  ✅ Saved daily_prices_retail_2020-03-30.csv (135 rows)
Fetching 2020-03-31 ...


/tmp/ipython-input-458471964.py:62: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)  # requires lxml


  ✅ Saved daily_prices_retail_2020-03-31.csv (135 rows)

Combined CSV written with 4185 total rows.

Combined CSV written with 4185 total rows.
